1. Validar la integridad referencial de los datos: registros de hechos (raw_fact) que no coincidan con sus dimensiones (raw_empleados, raw_productos, raw_locales).

In [0]:
%sql
SELECT 
    f.fecha_ventas,
    f.sku,
    f.id_vendedor,
    f.cantidad,
    e.id_vendedor AS existe_empleado,
    p.id_producto AS existe_producto,
    l.id_sucursal AS existe_sucursal
FROM bronze.raw_fact f
LEFT JOIN bronze.raw_empleados e
    ON f.id_vendedor = e.id_vendedor
LEFT JOIN bronze.raw_productos p
    ON f.sku = p.id_producto
LEFT JOIN bronze.raw_locales l
    ON e.sucursal = l.id_sucursal
WHERE e.id_vendedor IS NULL
   OR p.id_producto IS NULL
   OR l.id_sucursal IS NULL;


2.
Crear un esquema llamado “silver " en el catálogo “workspace”, crear dos tablas delta llamadas:
a.
“dim_vendedor” - Que unifique la información de vendedor con la de locales. La tabla debe tener las columnas:
i.
Id_vendedor
ii.
vendedor_nombre
iii.
sucursal_nombre

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.dim_vendedor AS
SELECT 
    e.id_vendedor,
    e.nombre AS vendedor_nombre,
    l.nombre AS sucursal_nombre
FROM bronze.raw_empleados e
LEFT JOIN bronze.raw_locales l
    ON e.sucursal = l.id_sucursal;


In [0]:
%sql
SELECT *
FROM workspace.silver.dim_vendedor
LIMIT 20;

b.
“dim_producto” – respetando la estructura la tabla equivalente (raw_productos).

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.dim_producto AS
SELECT 
    id_producto,
    familia,
    nombre,
    precio_unitario
FROM bronze.raw_productos;


In [0]:
%sql
SELECT *
FROM workspace.silver.dim_producto
LIMIT 20;

3.
Crear una tabla delta llamada “fact_ventas” asegurando:
a.
Integridad referencial entre la Fact y las dimensiones.
b.
Separar la fecha en los campos: dia, mes, ano

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.silver.fact_ventas AS
SELECT
    f.id_vendedor,
    f.sku        AS id_producto,
    f.cantidad,
    -- Descomposición de la fecha
    DAY(f.fecha_ventas)   AS dia,
    MONTH(f.fecha_ventas) AS mes,
    YEAR(f.fecha_ventas)  AS ano
FROM bronze.raw_fact f
INNER JOIN workspace.silver.dim_vendedor v
    ON f.id_vendedor = v.id_vendedor
INNER JOIN workspace.silver.dim_producto p
    ON f.sku = p.id_producto;


In [0]:
%sql
SELECT *
FROM workspace.silver.fact_ventas
LIMIT 20;